In [2]:
from biom import load_table
import pandas as pd
import numpy as np
from sklearn import linear_model
from scipy import stats
from itertools import product
import datetime
import os
import matplotlib.pyplot as plt

In [ ]:
class MiBiTimeSeries:
    # Class to handle time series data
    # Using numpy matrices for biom table instead of slow pandas dataframes

    def __init__(self, meta: pd.DataFrame, biom: pd.DataFrame, rared: int):
        # Initializer / Instance Attributes
        self.meta = meta  # Meta data (pandas DataFrame)
        self.biom = np.array(biom)  # Biological Observation Matrix (numpy array for faster computations)
        self.rared = rared  # Rarefaction depth
        self.all_corr = {}  # Placeholder for correlation results
        self.all_corr_thresh = {}  # Placeholder for thresholded correlation results
        self.otu_ids = 

    def _relative_abundance(self, node: int) -> np.array:
        """Calculate relative abundance of a given node."""
        return (self.biom[node] + 1e-6) / self.rared
    def lag_corr(self, i: int, j: int) -> tuple:
        # Function to perform a lagged correlation between two nodes (delta_i and j)
        # Compute relative abundance of node j
        j_relabund = self._relative_abundance(j)
        # Log-transform and remove first two elements from the series to eliminate effect of initial conditions
        j_term = np.log10(j_relabund)[2:]

        # Compute relative abundance of node i
        i_relabund = self._relative_abundance(i)
        # Compute change in log-transformed relative abundance of node i
        delta_i_term = np.log10(i_relabund)[2:] - np.log10(np.roll(i_relabund, 1))[2:]

        # Perform a Spearman rank correlation between j_term and delta_i_term
        mycorr = stats.spearmanr(j_term, delta_i_term)
        
        # Return the node pair, correlation coefficient, and p-value
        return i, j, *mycorr
    
    def plot_lag(self, nodes: tuple):
        # Function to plot a scatterplot of lagged relative abundance between two nodes (nodes[0] and nodes[1])
        i, j = nodes  # First node and second node

        # Compute log-transformed relative abundance of node j and remove first two elements
        j_term = np.log10(self._relative_abundance(j))[2:]

        # Compute change in log-transformed relative abundance of node i
        delta_i_term = np.log10(self._relative_abundance(i)[2:]) - np.log10(np.roll(self._relative_abundance(i), 1))[2:]
        # Plot a scatterplot of j_term vs delta_i_term
        plt.scatter(j_term, delta_i_term)
        
    def plot_abund(self, node: int):
        # Function to plot the abundance of a single node over time
        plt.plot(self.biom[node])  # Plot the abundance of the specified node
        
    def shuffle_biom(self):
        # Function to shuffle the columns of the biom DataFrame (presumably for permutation testing)
        np.random.shuffle(self.biom.T)  # Shuffle columns in place using numpy's function


In [ ]:
def meta_biom_filter(meta: pd.DataFrame, biom: pd.DataFrame, anon_name: str, filter_num=0):
    meta = meta.sort_index()
    
    meta = meta.loc[meta.ANONYMIZED_NAME == anon_name]
    meta = meta.loc[set(meta.index) & set(biom_df.columns)]
    biom_df = biom[list(set(meta.index) & set(biom_df.columns))]
    biom_df = biom_df.sort_index()
    biom_df = biom_df.loc[(biom_df.sum(axis=1) > filter_num)]
    
    avg_day = pd.DataFrame(index=biom_df.index)

    for i in set(meta.epoch_time):
        samples_day = meta.index[meta.epoch_time == i]
        avg_day[i] = biom_df.transpose().loc[samples_day].mean()
    
    #avg_day = avg_day.reindex_axis(sorted(avg_day.columns), axis=1)
    
    return(meta, avg_day)


In [ ]:
biom = load_table("./qiime_outputs/dada2_otu_table_w_tax_no_pynast_failures_rare7500.biom")
biom = biom.to_dataframe().transpose()
meta = pd.read_csv("./qiime_outputs/full_meta_w_times.csv")

In [ ]:
m01_meta = meta.loc[meta["ANONYMIZED_NAME"] == 'M01']
m01_meta = m01_meta.sort_values(["Time"])
m01_meta.index = m01_meta["#SampleID"]
m01_biom = biom.loc[m01_meta["#SampleID"]]


In [ ]:
df = m01_meta[["Time"]].merge(m01_biom, left_index=True, right_index=True)
grouped = df.groupby(["Time"]).sample()

In [ ]:
(grouped != 0).mean().sort_values().hist(bins=40)